In [1]:
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트

spark = (SparkSession.builder  # SparkSession 빌더 패턴 시작
         .appName("write-data")  # 애플리케이션 이름 설정
         .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
         .config("spark.executor.memory", "512m")  # Spark 설정 옵션
         .getOrCreate()  # SparkSession 생성 또는 기존 세션 반환)

spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/19 12:19:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 StructType, StructField, StringType, IntegerType, DateType

df = (spark.read.format("csv")  # CSV 형식으로 데이터 읽기
      .option("header", "true")  # 첫 번째 행을 헤더로 사용
      .option("nullValue", "null")
      .option("dateFormat",  # 날짜 형식 지정 "LLLL d, y")
      .load(  # 파일 로드"../data/netflix_titles.csv"))

In [7]:
(df.write.format("csv")
 .option("header", "true")  # 첫 번째 행을 헤더로 사용 
 .mode("overwrite")  # 기존 데이터 덮어쓰기
 .option("delimiter", ",")
 .save("../data/data_lake/netflix_csv_data"))

In [8]:
(df.write.format("json") 
 .mode("overwrite")  # 기존 데이터 덮어쓰기 
 .save("../data/data_lake/netflix_json_data"))

In [9]:
(df.write.format("parquet") 
 .mode("overwrite")  # 기존 데이터 덮어쓰기 
 .save("../data/data_lake/netflix_parquet_data"))

### Write Compressed Data

In [12]:
(df.write
 .format("csv")
 .mode("overwrite")  # 기존 데이터 덮어쓰기 
 .option("header", "true")  # 첫 번째 행을 헤더로 사용
 .option("delimiter", ",")
 .option("codec", "org.apache.hadoop.io.compress.GzipCodec")
 .save("../data/data_lake/netflix_csv_data.gz"))

### Specify the Number of Partitions

In [14]:
(df.repartition(  # 파티션 재분배4) 
 .write.format("csv") 
 .mode("overwrite")  # 기존 데이터 덮어쓰기 
 .option("header", "true")  # 첫 번째 행을 헤더로 사용 
 .option("delimiter", ",") 
 .save("../data/data_lake/netflix_csv_data_4_part")) 

### Use `coalesce()` to Reduce number fo Partitions

In [15]:
(df.coalesce(  # 파티션 수 줄이기1)
 .write.format("csv")
 .mode("overwrite")  # 기존 데이터 덮어쓰기 
 .option("header", "true")  # 첫 번째 행을 헤더로 사용
 .option("delimiter", ",")
 .save("../data/data_lake/netflix_csv_data_whole"))

### Use `partitionBy()` to write partitions based on a column

In [16]:
# partition the CSV data by the 'release_year' column
(df.write.format('csv')
 .option("header", "true")  # 첫 번째 행을 헤더로 사용
 .option('delimiter', ',')
 .mode("overwrite")  # 기존 데이터 덮어쓰기
 .partitionBy('release_year')
 .save("../data/data_lake/netflix_csv_data_partitioned"))

In [17]:
spark.stop()  # Spark 세션 종료 - 리소스 정리